# ETL - Criaturas das coleções Core e Expansion de Magic The Gathering

## Introdução

Magic The Gathering é um jogo de cartas colecionáveis criado por Richard Garfield, lançado em 1993 pela *Wizards of the Coast*. É o primeiro jogo de cartas colecionáveis da história e hoje conta com cerca de 50 milhões de jogadores em todo o mundo.

O jogo consiste em diferentes tipos de cartas com efeitos que interagem entre si, criando uma gameplay complexa e envolvente. Cada jogador possui um conjunto de cartas chamado de grimório ou *deck*, que contém as cartas disponíveis para aquele jogador em uma partida.

O jogo sofreu leve modificações e atualizações desde suas versões iniciais, se consolidando no jogo que é hoje. Os tipos de cartas que se pode encontrar no jogo são: Criaturas, Artefatos, Mágicas Instantâneas, Feitiços, Terrenos e Planeswalkers.

Desde sua primeira coleção lançada, conhecida como *Limited Edition Alpha*, foram lançadas mais de 900 outras coleções, aumentando consideravelmente o número de cartas no jogo. Até a data desta ETL, a última coleção lançada foi *Spotlight Series*, em janeiro de 2025.

Uma das formas que os jogadores e colecionadores podem procurar por cartas do jogo e buscar por cartas que ainda serão lançadas é através da plataforma [Scryfall](https://scryfall.com/), que oferece a API que será utilizada neste projeto.

## Objetivo

Dentre as coleções lançadas, há coleções voltadas para formatos de jogo espefícicos, como as coleções focadas nos formatos Modern ou Commander, e há também coleções com cartas com finalidade humorísticas que não são válidas para jogo. Entretanto, há dois tipos de coleções que consideramos como básicos nesta ETL: as coleções do tipo *Core*, que são lançadas anualmente, como as coleções "Tenth Edition" ou "M15"; e as coleções do tipo *Expansion*, que são as popularmente conhecidas como coleções de bloco, que possuem cartas que se relacionam dentro de uma mesma *lore*.

Nosso objetivo nesta ETL é utilizar a API do Scryfall para obter todas as cartas do tipo Criatura dentro das coleções básicas do jogo (coleções *Core* e *Expansions*) e armazenar as suas informações básicas em um arquivo CSV.

As informações básicas que vamos considerar são: nome da carta, nome da coleção, valor de mana (custo convertido de mana - CMC), poder, resistência e texto da carta (texto de oráculo).



## Referências

[Magic: The Gathering - Wikipedia (EN)](https://en.wikipedia.org/wiki/Magic:_The_Gathering)

[Scryfall Magic The Gathering Search](https://scryfall.com/)

[REST API Documentation - Scryfall](https://scryfall.com/docs/api)

# Extração

## I. Extração das coleções do tipo *Core* e *Expansion* de Magic The **Gathering**

In [ ]:
import requests
import json
import time
import pandas as pd

# API URL
scryfall_api_url = "https://api.scryfall.com"

In [ ]:
# Definimos essa função para fazer requisições pois a documentação da API do
# Scryfall pede por um delay de 50-100 ms entre requisições consecutivas. Desta
# forma, podemos garantir que o delay será respeitado
def request_get(url):
    # Mede o tempo antes
    before_time = time.time()

    # Faz requisição
    response = requests.get(url)

    # Mede o tempo depois
    after_time = time.time()

    # Calcula o tempo que a requisição levou
    ellapsed_time = after_time - before_time

    # Se a requisição demorou menos de 100 ms:
    if ellapsed_time < 0.1:
        # Calcula o tempo restante de espera
        wait_time = 0.1 - ellapsed_time

        # Espera o tempo restante para fazer a requisição demorar 100 ms
        time.sleep(wait_time)

    # Retorna a resposta
    return response

In [ ]:
# Pedindo todas as coleções de MTG
response = request_get(f"{scryfall_api_url}/sets")

# Obtém o campo 'data', que é uma lista de coleções
mtg_sets = response.json()['data']

# Tamanho da lista (quantidade de coleções)
print(f"Total de coleções obtidas: {len(mtg_sets)}")

Total de coleções obtidas: 940


In [ ]:
# Vamos selecionar apenas as coleções 'core' ou 'expansions'
mtg_basic_sets = [mtg_set for mtg_set in mtg_sets if (mtg_set['set_type'] in ['core', 'expansion'])]

# Tamanho da lista (quantidade de coleções)
print(f"Total de coleções selecionadas: {len(mtg_basic_sets)}")

Total de coleções selecionadas: 131


### Exemplo de um objeto *set*

In [ ]:
print(json.dumps(mtg_basic_sets[42], indent=2))

{
  "object": "set",
  "id": "0eeb9a9a-20ac-404d-b55f-aeb7a43a7f62",
  "code": "ori",
  "mtgo_code": "ori",
  "arena_code": "ori",
  "tcgplayer_id": 1512,
  "name": "Magic Origins",
  "uri": "https://api.scryfall.com/sets/0eeb9a9a-20ac-404d-b55f-aeb7a43a7f62",
  "scryfall_uri": "https://scryfall.com/sets/ori",
  "search_uri": "https://api.scryfall.com/cards/search?include_extras=true&include_variations=true&order=set&q=e%3Aori&unique=prints",
  "released_at": "2015-07-17",
  "set_type": "core",
  "card_count": 288,
  "printed_size": 272,
  "digital": false,
  "nonfoil_only": false,
  "foil_only": false,
  "block_code": "lea",
  "block": "Core Set",
  "icon_svg_uri": "https://svgs.scryfall.io/sets/ori.svg?1736139600"
}


## II. Obtenção das cartas de cada coleção

In [ ]:
# Obtem o conjunto de cartas de uma coleção dada
def getSetCards(mtg_set):
    # Pedido pelas cartas da coleção dada
    response = request_get(mtg_set['search_uri'])

    cards = response.json()['data']

    # Obtem páginas faltantes se houver
    while response.json()['has_more']:
        response = request_get(response.json()['next_page'])

        # Obtem a lista de cartas da nova resposta
        cards += response.json()['data']

    # Seleciona apenas as cartas de papel
    paper_cards = [card for card in cards if 'paper' in card['games']]

    # Faz log das informações
    log_msg = f"Obtivemos {len(cards)} cartas do conjunto {mtg_set['name']}. Destas cartas, obtivemos {len(paper_cards)} cartas no formato 'papel'.\n"

    return paper_cards, log_msg

# Obtenção de todas as cartas das coleções com log das operações
cards = []

# Como esta é uma requisição demorada, vamos utilizar variável que dirá quando
# vamos imprimir o progresso da requisição
next_print = 10

# Abre arquivo criado para realizar o log
with open("card_extraction.txt","w") as log_file:

    # Imprime progresso inicial
    print(f"Progresso: 0/{len(mtg_basic_sets)} coleções (0%)")

    # Anota o tempo inicial
    start_time = time.time()

    # Percorre as coleções, obtendo as cartas de cada uma
    for i, mtg_set in enumerate(mtg_basic_sets):
        new_cards, log_msg = getSetCards(mtg_set)

        # Adiciona as cartas na lista
        cards += new_cards

        # Adiciona o log no arquivo
        log_file.write(log_msg)

        # Calcula tempo passado desde o início do processo:
        ellapsed_time = time.time() - start_time

        # Utiliza i para imprimir uma barra de progresso
        if ellapsed_time > next_print:
            next_print += 10
            progress = 100 * (i+1) / len(mtg_basic_sets)
            print(f"Progresso: {i+1}/{len(mtg_basic_sets)} coleções ({progress:.1f}%)")

    # Imprime última mensagem de progresso:
    print(f"Progresso: {len(mtg_basic_sets)}/{len(mtg_basic_sets)} coleções (100%)")
    print(f"\nTempo de conclusão: {time.time() - start_time:.1f} segundos.")

    # Imprime total de cartas obtidas e adiciona ao log
    log_msg = f"\nTotal de cartas: {len(cards)}"
    print(log_msg)
    log_file.write(log_msg)

Progresso: 0/131 coleções (0%)
Progresso: 13/131 coleções (9.9%)
Progresso: 25/131 coleções (19.1%)
Progresso: 43/131 coleções (32.8%)
Progresso: 61/131 coleções (46.6%)
Progresso: 83/131 coleções (63.4%)
Progresso: 101/131 coleções (77.1%)
Progresso: 118/131 coleções (90.1%)
Progresso: 131/131 coleções (100%)

Tempo de conclusão: 77.7 segundos.

Total de cartas: 36468


### Exemplo de um objeto *card*

In [ ]:
print(json.dumps(cards[1337], indent=2))

{
  "object": "card",
  "id": "58706bd8-558a-43b9-9f1e-c1ff0044203b",
  "oracle_id": "b0747a4f-d041-4750-aef5-a5d101696f65",
  "multiverse_ids": [
    669087
  ],
  "mtgo_id": 129591,
  "arena_id": 91709,
  "tcgplayer_id": 558693,
  "cardmarket_id": 777777,
  "name": "Galewind Moose",
  "lang": "en",
  "released_at": "2024-08-02",
  "uri": "https://api.scryfall.com/cards/58706bd8-558a-43b9-9f1e-c1ff0044203b",
  "scryfall_uri": "https://scryfall.com/card/blb/173/galewind-moose?utm_source=api",
  "layout": "normal",
  "highres_image": true,
  "image_status": "highres_scan",
  "image_uris": {
    "small": "https://cards.scryfall.io/small/front/5/8/58706bd8-558a-43b9-9f1e-c1ff0044203b.jpg?1721426814",
    "normal": "https://cards.scryfall.io/normal/front/5/8/58706bd8-558a-43b9-9f1e-c1ff0044203b.jpg?1721426814",
    "large": "https://cards.scryfall.io/large/front/5/8/58706bd8-558a-43b9-9f1e-c1ff0044203b.jpg?1721426814",
    "png": "https://cards.scryfall.io/png/front/5/8/58706bd8-558a-43b9-

# Transformação

## I. Separação das criaturas

In [ ]:
# Verifica se uma carta é uma criatura
def is_creature(card):
    return 'Creature' in card['type_line']

# Cria lista de cartas de criatura
creature_cards = [card for card in cards if is_creature(card)]

# Imprime informação
print(f"Das {len(cards)} cartas, obtivemos {len(creature_cards)} cartas de criatura.")

Das 36468 cartas, obtivemos 18095 cartas de criatura.


## II. Conversão em DataFrame

In [ ]:
0# Colunas do DataFrame
df_columns = ['name', 'set_name', 'cmc', 'power', 'toughness', 'oracle_text']

# Obtenção da linha do dataframe a partir de um objeto de carta
def get_row(card):
    return [card[item] if item in card else None for item in df_columns]

# Criação do conteúdo do DataFrame
df_rows = [get_row(card) for card in creature_cards]

# Criação do DataFrame
df = pd.DataFrame(df_rows, columns=df_columns)

df.head()

,name,set_name,cmc,power,toughness,oracle_text
0,"Daretti, Rocketeer Engineer",Aetherdrift,5.0,*,5,Daretti's power is equal to the greatest mana ...
1,Brightglass Gearhulk,Aetherdrift,4.0,4,4,"First strike, trample\nWhen this creature ente..."
2,Sire of Seven Deaths,Foundations,7.0,7,7,"First strike, vigilance\nMenace, trample\nReac..."
3,"Arahbo, the First Fang",Foundations,3.0,2,2,Other Cats you control get +1/+1.\nWhenever Ar...
4,Armasaur Guide,Foundations,5.0,4,4,Vigilance (Attacking doesn't cause this creatu...


In [ ]:
# Shape do DataFrame
df.shape

(18095, 6)

# Carregamento

In [ ]:
# Salvamos o DataFrame como um arquivo CSV
df.to_csv('basic_creatures.csv', sep=';')